In [1]:
# add path to explore_by_example src folder
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


# import usual libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import f1_score

# our system imports
from aideme import *

%matplotlib inline

In [2]:
def plot_fscore(metrics):
    df_list = [pd.DataFrame.from_dict({i: metric for i, metric in enumerate(ls)}, orient='index') for ls in metrics]
    avg = sum([df['fscore'][~df['fscore'].isna()] for df in df_list]) / len(df_list)
    avg.plot(ylim=[0,1], marker='o')
    plt.show()

In [3]:
# DUMMY DATA
N = int(1e5)
dim = 2
limit = 2 * (0.001)**(1. / dim)
rng = np.random.RandomState(0)
X = rng.uniform(low=-2, high=2, size=(N, dim))  # do not forget to standardize the data. For this distribution, it should be fine without it.
y_subspace = np.vstack([np.abs(X[:, i]) < limit for i in range(dim)]).T.astype('float')  # partial labels (for each subspace)
y = y_subspace.min(axis=1)
labeled_set = LabeledSet(y)

# visualize data distribution
print('selectivity :', 100 * y.sum() / len(y), '%')

#plt.figure(figsize=(10,8))
#plt.scatter(X[:, 0], X[:, 1], s=0.5, c=['b' if lb else 'r' for lb in y])
#plt.show()

selectivity : 0.105 %


# NO FACTORIZATION EXAMPLE

In [4]:
# SET-UP EXPLORATION CONFIGURATION
NUMBER_OF_ITERATIONS = 100  # number of points to be labeled by the user
SUBSAMPLING = None
INITIAL_SAMPLER = stratified_sampler(labeled_set, pos=1, neg=1)  # start with one random positive sample and one random negative sample
#INITIAL_SAMPLER = random_sampler(10)
CALLBACK = [ # callback functions to be called at the end of each iteration
    classification_metrics(y, 'fscore'), 
    three_set_metric,
]
CALLBACK_SKIP = 10
PRINT_CALLBACK_RESULT = True
CONVERGENCE_CRITERIA = [
    max_iter_reached(NUMBER_OF_ITERATIONS),
    #all_points_are_known,
    #metric_reached_threshold('fscore', 0.8),
    #metric_reached_threshold('tsm', 0.9),
]

explore = PoolBasedExploration(INITIAL_SAMPLER, SUBSAMPLING, CALLBACK, CALLBACK_SKIP, PRINT_CALLBACK_RESULT, CONVERGENCE_CRITERIA)

# CHOOSE AN ALGORITHM
#learner = RandomSampler(SVC(C=1e5, kernel='rbf', gamma='auto'))  # choose a random point
learner = SimpleMargin(C=1024, kernel='rbf')  # choose point closest to SVM decision boundary
#learner = KernelQueryByCommittee(kernel='rbf', sampling='deterministic', n_samples=8, warmup=1000, thin=100, rounding=True)  # version space algorithm
learner = DualSpaceModel(learner, sample_unknown_proba=0.5, mode='positive')  # Dual Space model


# RUN EXPLORATION
metrics = explore.run(X, labeled_set, learner, repeat=1)  # 'repeat' specifies how many times to repeat the exploration process

# COMPUTE AVERAGE F-SCORE OVER ALL REPEATS AND PLOT
#plot_fscore(metrics)
df = pd.DataFrame.from_dict({i: metric for i, metric in enumerate(metrics[0])}, orient='index')
df

iter: 1, fscore: 0.002922633710492255, tsm: 0.0
iter: 11, fscore: 0.03655988857938719, tsm: 0.0
iter: 21, fscore: 0.16921837228041903, tsm: 0.0
iter: 31, fscore: 0.348993288590604, tsm: 7.001575354454752e-05
iter: 41, fscore: 0.5211267605633803, tsm: 0.0020216512325551064
iter: 51, fscore: 0.8898678414096917, tsm: 0.09102902374670185
iter: 61, fscore: 0.7950819672131147, tsm: 0.31716417910447764
iter: 71, fscore: 0.9509803921568627, tsm: 0.636986301369863
iter: 81, fscore: 0.9389671361502347, tsm: 0.776
iter: 91, fscore: 0.9805825242718447, tsm: 0.9181818181818182
iter: 101, fscore: 1.0, tsm: 1.0


,phase,iter_time,labeled_indexes,final_labels,partial_labels,fit_time,get_next_time,fscore,tsm
0,begin,0.002202,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,initial_sampling,0.034294,"[49136, 643]","[1.0, 0.0]","[[1.0], [0.0]]",0.000958,0.033337,0.002923,0.0
2,exploration,0.034029,[25811],[0.0],[[0.0]],0.001019,0.033009,NaN,NaN
3,exploration,0.035464,[58486],[0.0],[[0.0]],0.000787,0.034677,NaN,NaN
4,exploration,0.039317,[15422],[0.0],[[0.0]],0.000759,0.038558,NaN,NaN
...,...,...,...,...,...,...,...,...,...
97,exploration,0.322860,[99053],[1.0],[[1.0]],0.010575,0.312286,NaN,NaN
98,exploration,0.158331,[82759],[0.0],[[0.0]],0.002812,0.155519,NaN,NaN
99,exploration,0.172217,[61124],[1.0],[[1.0]],0.009904,0.162313,NaN,NaN
100,exploration,0.174691,[75881],[1.0],[[1.0]],0.011188,0.163502,NaN,NaN


# FACTORIZATION EXAMPLE

In [17]:
# SET-UP EXPLORATION CONFIGURATION
NUMBER_OF_ITERATIONS = 100  # number of points to be labeled by the user
SUBSAMPLING = None
INITIAL_SAMPLER = stratified_sampler(labeled_set, pos=1, neg=1)  # start with one random positive sample and one random negative sample
#INITIAL_SAMPLER = random_sampler(10)
CALLBACK = [ # callback functions to be called at the end of each iteration
    classification_metrics(y, 'fscore'), 
    three_set_metric,
]
CALLBACK_SKIP = 10
PRINT_CALLBACK_RESULT = True
CONVERGENCE_CRITERIA = [
    max_iter_reached(NUMBER_OF_ITERATIONS),
    #all_points_are_known,
    #metric_reached_threshold('fscore', 0.8),
    metric_reached_threshold('tsm', 0.9),
]

explore = PoolBasedExploration(INITIAL_SAMPLER, SUBSAMPLING, CALLBACK, CALLBACK_SKIP, PRINT_CALLBACK_RESULT, CONVERGENCE_CRITERIA)

# Factorized AL algorithms
PARTITION = [[i] for i in range(dim)]  #[[0], [1]]

# FACTORIZED VERSION SPACE
# label_function = 'AND', 'OR', 'PROD'
# loss = 'GREEDY', 'SQUARED', 'PRODUCT'
#learner = SubspatialVersionSpace(warmup=100, thin=10, n_samples=8, rounding=True, kernel='rbf', gamma=None, partition=PARTITION, label_function='AND', loss='GREEDY')

# FACTORIZED SIMPLE MARGIN
#learner = SubspatialSimpleMargin(C=1024, kernel='rbf', gamma=5, partition=PARTITION, label_function='AND')

# FACTORIZED DSM
learner = DualSpaceModel(SimpleMargin(C=1024, kernel='rbf'), mode='positive', sample_unknown_proba=0.5, partition=PARTITION)  # Dual Space model


# RUN EXPLORATION
metrics = explore.run(X, labeled_set, learner, repeat=1)  # use y_subspace here

# COMPUTE AVERAGE F-SCORE OVER ALL REPEATS AND PLOT
#plot_fscore(metrics)
df = pd.DataFrame.from_dict({i: metric for i, metric in enumerate(metrics[0])}, orient='index')
df

iter: 1, fscore: 0.00451292631036039, tsm: 0.0
iter: 11, fscore: 0.08765652951699464, tsm: 0.024150634465820713
iter: 21, fscore: 0.8586956521739131, tsm: 0.5851851851851851
iter: 31, fscore: 0.976303317535545, tsm: 0.9090909090909091


,phase,iter_time,labeled_indexes,final_labels,partial_labels,fit_time,get_next_time,fscore,tsm
0,begin,0.002335,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,initial_sampling,0.505892,"[20903, 76449]","[1.0, 0.0]","[[1.0, 1.0], [0.0, 0.0]]",0.472531,0.033361,0.004513,0.000000
2,exploration,0.156568,[31853],[0.0],"[[0.0, 0.0]]",0.144258,0.012310,NaN,NaN
3,exploration,0.172571,[54432],[0.0],"[[0.0, 0.0]]",0.137219,0.035351,NaN,NaN
4,exploration,0.175546,[72349],[0.0],"[[0.0, 0.0]]",0.136760,0.038786,NaN,NaN
5,exploration,0.119136,[45116],[0.0],"[[0.0, 0.0]]",0.003746,0.115390,NaN,NaN
6,exploration,0.020870,[80475],[0.0],"[[0.0, 1.0]]",0.018564,0.002305,NaN,NaN
7,exploration,0.009547,[77995],[0.0],"[[1.0, 0.0]]",0.007617,0.001930,NaN,NaN
8,exploration,0.041746,[8751],[0.0],"[[0.0, 1.0]]",0.002036,0.039711,NaN,NaN
9,exploration,0.120420,[56064],[0.0],"[[0.0, 0.0]]",0.006576,0.113844,NaN,NaN
